In [1]:
import json
import glob 
import os
import pandas as pd
from PIL import Image
import base64
import io
from openai import OpenAI
from pydantic import BaseModel
from textwrap import dedent
from tqdm import tqdm, trange

client = OpenAI(timeout=20)

def base64_to_image(base64_str):
    image_data = base64.b64decode(base64_str)
    image = Image.open(io.BytesIO(image_data))
    return image

# annotation_dir = "/pasteur2/u/suyc/VLMEval/AutoConverter/visualization"
# dataset_dir = "/pasteur2/u/suyc/VLMEval/VLMEvalKit/LMUData/"

# annotations = []
# for filename in glob.glob(annotation_dir + "/*/*.xlsx"):
#     print(filename)
#     basename = os.path.basename(filename)
#     data = pd.read_csv(dataset_dir + basename.replace('_correctness.xlsx', '-50.tsv'), sep='\t').to_dict('records')[:50]
#     annotation = pd.read_excel(filename).to_dict('records')[:50]
#     for item, anno in zip(data, annotation):
#         anno["filename"] = basename
#         anno["image"] = item["image"]
#     annotations += annotation

# # json.dump(annotations, open('annotations_20241109_0042.json', 'w'), indent=4)
# display(len(annotations), annotations[0])
# display(base64_to_image(annotations[0]["image"]))

annotations = json.load(open('annotations_20241109_0042.json'))

In [2]:
# class Judgement(BaseModel):
#     reasoning: str
#     correctness: int


# def judge_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
#     system_prompt = f"""
#     Your task is to evaluate a multiple-choice question (with accompanying image) to determine if any incorrect choices (distractors) could also be considered correct answers.

#     CRITICAL: The marked correct answer MUST always be treated as valid and correct, regardless of your own assessment. Never question or evaluate the correct answer - your task is to accept it as an absolute truth and evaluate only whether other choices could also be correct.

#     Score the question's correctness using this scale:
#     5 - Perfect: All other choices are clearly incorrect
#     4 - Good: Other choices are mostly wrong but have minor elements of correctness
#     3 - Fair: At least one other choice could be partially correct
#     2 - Poor: At least one other choice could be equally correct
#     1 - Invalid: Multiple choices are equally valid as the correct answer

#     Provide:
#     1. Score (1-5)
#     2. Brief explanation focusing specifically on any problematic distractor choices

#     Remember: Never analyze whether the marked correct answer is right or wrong - it is ALWAYS correct by definition. Focus exclusively on whether other choices could also be valid answers.
#     """

#     prompt = f"""
#     Question: {question}
#     Choices: {choices}
#     Correct Answer: {correct_choice}
#     """

#     response = client.beta.chat.completions.parse(
#         model="gpt-4o", 
#         messages=[
#             {"role": "system", "content": dedent(system_prompt)}, # "You are a helpful assistant."
#             {"role": "user", "content": [
#                 {
#                     "type": "text",
#                     "text": dedent(prompt)
#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": f"data:image/jpeg;base64,{image_base64}"
#                     }
#                 }
#             ]}
#         ],
#         response_format=Judgement,
#         temperature=0  # Set to 0 for deterministic responses
#     )
    
#     answer = response.choices[0].message.parsed.dict()
#     return answer



class Judgement(BaseModel):
    reasoning: str
    correctness: int
    improvement: str
    
def judge_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
    system_prompt = f"""
    Your task is to evaluate a multiple-choice question (with accompanying image) to determine if any incorrect choices (distractors) could also be considered correct answers.

    CRITICAL: The marked correct answer MUST always be treated as valid and correct, regardless of your own assessment. Never question or evaluate the correct answer - your task is to accept it as an absolute truth and evaluate only whether other choices could also be correct.

    Score the question's correctness using this scale:
    5 - Perfect: All other choices are clearly incorrect
    4 - Good: Other choices are mostly wrong but have minor elements of correctness
    3 - Fair: At least one other choice could be partially correct
    2 - Poor: At least one other choice could be equally correct
    1 - Invalid: Multiple choices are equally valid as the correct answer

    Provide:
    1. Score (1-5)
    2. Brief explanation focusing specifically on any problematic distractor choices
    3. Suggested improvements for the problematic distractors (if applicable)

    Remember: Never analyze whether the marked correct answer is right or wrong - it is ALWAYS correct by definition. Focus exclusively on whether other choices could also be valid answers.
    """

    prompt = f"""
    Question: {question}
    Choices: {choices}
    Correct Answer: {correct_choice}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": dedent(system_prompt)}, # "You are a helpful assistant."
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]}
        ],
        response_format=Judgement,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    answer = response.choices[0].message.parsed.dict()
    return answer



In [3]:
item = annotations[0]
prediction = judge_multichoice_correctness_with_image(item["image"], item["question"], [item["A"], item["B"], item["C"], item["D"]], item[item["answer"]])
display(prediction)

{'reasoning': 'The image shows a variety of vases and similar objects that appear to be made from clay, which aligns with pottery. However, ceramics is a broader term that includes pottery, so it could also be considered partially correct. Glassblowing and sculpture are clearly incorrect as the objects are not made of glass and do not represent typical sculptures.',
 'correctness': 3,
 'improvement': "Replace 'ceramics' with a more distinct option like 'painting' to avoid overlap with 'pottery'."}

In [4]:
# predictions = []
# for item in tqdm(annotations):
#     prediction = judge_multichoice_correctness_with_image(item["image"], item["question"], [item["A"], item["B"], item["C"], item["D"]], item[item["answer"]])
#     predictions.append(prediction)


from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def process_item(item):
    return judge_multichoice_correctness_with_image(
        item["image"],
        item["question"],
        [item["A"], item["B"], item["C"], item["D"]],
        item[item["answer"]]
    )

def parallel_judge(annotations):
    with ProcessPoolExecutor(max_workers=32) as executor:
        results = list(tqdm(executor.map(process_item, annotations), total=len(annotations)))
    return results

predictions = parallel_judge(annotations)

100%|██████████| 450/450 [01:01<00:00,  7.31it/s]


In [5]:
json.dump(predictions, open('predictions_20241109_2150_final.json', 'w'), indent=4)

In [7]:
import json

predictions = json.load(open('predictions_20241109_2150_final.json'))
annotations = json.load(open('annotations_20241109_0042.json'))

preds = [item["correctness"] for item in predictions]
labels = [item["correctness"] for item in annotations]

# compute accuracy, f1, auroc
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_fscore_support


auroc = roc_auc_score(labels, preds)
preds_binary = [1 if pred >= 2 else 0 for pred in preds]

accuracy = accuracy_score(labels, preds_binary)
p, r, f1, _ = precision_recall_fscore_support(labels, preds_binary, average=None)
precision = p[0]
recall = r[0]
f1 = f1[0]


print("AUROC:", auroc)
print("Accuracy:", accuracy)
print("Precision", precision)
print("Recall", recall)
print("F1:", f1)

# if preds==1, print the percentage of questions are indeed wrong
for i in range(1, 6):
    selected_preds = [label for label, pred in zip(labels, preds) if pred==i]
    print(i, selected_preds.count(0), len(selected_preds))


AUROC: 0.7670507544581618
Accuracy: 0.8777777777777778
Precision 0.1875
Recall 0.06666666666666667
F1: 0.09836065573770492
1 3 16
2 12 23
3 12 67
4 7 26
5 11 318


In [ ]:
for idx, (pred, anno) in enumerate(zip(predictions, annotations)):
    if pred["correctness"] == 1:  # and anno["correctness"] == 0
        display(idx)
        display(anno["question"], [anno["A"], anno["B"], anno["C"], anno["D"]], anno[anno["answer"]]) # , pred["reasoning"])
        image = base64_to_image(anno["image"])
        display(image)

In [ ]:
# for pred, anno in zip(predictions, annotations):
#     display(pred)
#     display(anno)
#     image = base64_to_image(anno["image"]).resize((384, 384))
#     display(image)
#     input()

# Examine Data

In [ ]:
data = pd.read_csv("/pasteur2/u/suyc/VLMEval/VLMEvalKit/LMUData/VMCBench-1000-v2.tsv", sep='\t').to_dict('records')

In [ ]:
data[336]

# Refine Captialization

In [ ]:
class Question(BaseModel):
    distractors: list[str]

client = OpenAI(timeout=20)

def refine_question_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
    system_prompt = f"""
    Your task is to refine the distractors (incorrect options) of a multiple-choice question by matching their capitalization style with the correct answer:

    1. Analyze the capitalization pattern of the correct answer (e.g., "Full Capitals", "first Letter Only", "all lowercase", "camelCase", "Title Case")
    2. Modify each distractor to follow the same capitalization pattern as the correct answer
    - Every word position should match (first word, middle words, last word)
    - Special terms, acronyms, and proper nouns should maintain their standard capitalization
    3. Do not make any other changes to the distractors' content, length, or meaning
    4. Do not modify the correct answer or question itself

    If the distractors already match the correct answer's capitalization pattern, leave them unchanged.

    Please output only the refined distractors.
    """

    prompt = f"""
    Correct Answer: ```{correct_choice}```
    Distractors: ```{list(set(choices) - set([correct_choice]))}```
    """
    print(prompt)

    response = client.beta.chat.completions.parse(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": dedent(system_prompt)}, # "You are a helpful assistant."
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                # {
                #     "type": "image_url",
                #     "image_url": {
                #         "url": f"data:image/jpeg;base64,{image_base64}"
                #     }
                # }
            ]}
        ],
        response_format=Question,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    answer = response.choices[0].message.parsed.dict()
    return answer



In [ ]:
item = annotations[103] # 3, 103, 200
prediction = refine_question_with_image(item["image"], item["question"], [item["A"], item["B"], item["C"], item["D"]], item[item["answer"]])
display(prediction)

# Correct Question

In [ ]:
class Judgement(BaseModel):
    reasoning: str
    correctness: int
    improvement: str


class Question(BaseModel):
    reasoning: str
    distractors: list[str]


client = OpenAI(timeout=20)

def judge_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str) -> str:
    system_prompt = f"""
    Your task is to evaluate a multiple-choice question (with accompanying image) to determine if any incorrect choices (distractors) could also be considered correct answers.

    CRITICAL: The marked correct answer MUST always be treated as valid and correct, regardless of your own assessment. Never question or evaluate the correct answer - your task is to accept it as an absolute truth and evaluate only whether other choices could also be correct.

    Score the question's correctness using this scale:
    5 - Perfect: All other choices are clearly incorrect
    4 - Good: Other choices are mostly wrong but have minor elements of correctness
    3 - Fair: At least one other choice could be partially correct
    2 - Poor: At least one other choice could be equally correct
    1 - Invalid: Multiple choices are equally valid as the correct answer

    Provide:
    1. Score (1-5)
    2. Brief explanation focusing specifically on any problematic distractor choices
    3. Suggested improvements for the problematic distractors (if applicable)

    Remember: Never analyze whether the marked correct answer is right or wrong - it is ALWAYS correct by definition. Focus exclusively on whether other choices could also be valid answers.
    """

    prompt = f"""
    Question: {question}
    Choices: {choices}
    Correct Answer: {correct_choice}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": dedent(system_prompt)}, # "You are a helpful assistant."
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]}
        ],
        response_format=Judgement,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    answer = response.choices[0].message.parsed.dict()
    return answer




def improve_multichoice_correctness_with_image(image_base64: str, question: str, choices: list, correct_choice: str, issue: str, improvement: str) -> str:
    system_prompt = """
    You are an expert in educational assessment design specializing in multiple-choice question improvement. Your task is to enhance question effectiveness by revising problematic distractors (incorrect answer choices) while maintaining the existing correct answer.

    Input Required:
    1. The complete question
    2. The current correct answer
    3. Any associated images/materials
    4. Specific feedback about problematic distractors
    5. Suggested improvements (if provided)

    Analysis Steps:
    1. Review the question content and learning objective
    2. Analyze the designated correct answer
    3. Examine the feedback regarding problematic distractors
    4. Evaluate any provided suggestions for improvement:
    - Assess if suggestions fully address the identified issues
    - Determine if suggestions align with best practices
    - Identify any gaps or weaknesses in the suggestions
    5. Develop exactly 3 improved distractors that:
    - Are plausible but clearly incorrect
    - Address the identified issues
    - Align with common student misconceptions
    - Maintain consistent format and length with other options
    - Go beyond provided suggestions when necessary for better quality

    Guidelines:
    1. Treat the marked correct answer as fixed and unchangeable
    2. Only modify distractors specifically identified as problematic
    3. Preserve any well-functioning distractors
    4. Maintain the original difficulty level of the question
    5. Use your expertise to improve upon or deviate from provided suggestions if they:
    - Are too vague or incomplete
    - Don't fully address the identified issues
    - Could be enhanced for better assessment quality
    - Miss important misconceptions or learning opportunities

    Output:
    1. Brief analysis of the distractor issues and improvement approach
    2. Three improved distractors
    """

    prompt = f"""
    Question: {question}
    Choices: {choices}
    Correct Answer: {correct_choice}
    Identified Issues: {issue}
    Suggested Improvements: {improvement}
    """

    print(prompt)

    response = client.beta.chat.completions.parse(
        model="gpt-4o", 
        messages=[
            {"role": "system", "content": dedent(system_prompt)}, # "You are a helpful assistant."
            {"role": "user", "content": [
                {
                    "type": "text",
                    "text": dedent(prompt)
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}"
                    }
                }
            ]}
        ],
        response_format=Question,
        temperature=0  # Set to 0 for deterministic responses
    )
    
    distractors = response.choices[0].message.parsed.dict()
    return distractors

In [ ]:
from copy import deepcopy
import random

item = deepcopy(annotations[373])
display(item)

################ Round 1 ################
judgement = judge_multichoice_correctness_with_image(item["image"], item["question"], [item["A"], item["B"], item["C"], item["D"]], item[item["answer"]])
display(judgement)
distractors = improve_multichoice_correctness_with_image(item["image"], item["question"], [item["A"], item["B"], item["C"], item["D"]], item[item["answer"]], judgement["reasoning"], judgement["improvement"])
display(distractors)
answer = item[item["answer"]]
options = distractors["distractors"] + [answer]
random.shuffle(options)
item["A"], item["B"], item["C"], item["D"] = options
item["answer"] = "ABCD"[options.index(answer)]
display("-" * 63)

################ Round 2 ################
# ...
